In [ ]:
# Load and combine Syn, Ldap and NetBios data
import pandas as pd
import numpy as np

chunksize = 10 ** 5

synData = pd.DataFrame()
ldapData = pd.DataFrame()
netbiosData = pd.DataFrame()
data = pd.DataFrame()

for chunk in pd.read_csv("data/03-11/Syn.csv", chunksize=chunksize, nrows=1000000):
    synData = synData.append(chunk)

data = data.append(synData)
del synData

for chunk in pd.read_csv("data/03-11/LDAP.csv", chunksize=chunksize, nrows=1000000):
    ldapData = ldapData.append(chunk)

data = data.append(ldapData)
del ldapData

for chunk in pd.read_csv("data/03-11/NetBIOS.csv", chunksize=chunksize, nrows=1000000):
    netbiosData = netbiosData.append(chunk)

data = data.append(netbiosData)
del netbiosData

# - - - - - - - - - -
# Drop NaN and Inf values

data.replace([np.inf, -np.inf], np.nan, inplace=True)
data = data.dropna()

# # - - - - - - - - - -
# Converting data to the right floats, removing unecessary fields
# Convert int64 and str to float 64

import ipaddress

data.replace({'Syn': 1, 'NetBIOS': 1, 'LDAP': 1, 'BENIGN': 0}, inplace=True) # Replace strings
data[' Label'] = data[' Label'].astype(np.float64) # Cast from int64 to float 64

data['SimillarHTTP'] = data['SimillarHTTP'].astype(bool).astype(np.float64) # Replace non-zero with 1

data.drop(['Unnamed: 0'], axis=1, inplace=True) # drop Unnamed: 0 because is just an ID
data.drop(['Flow ID'], axis=1, inplace=True) # drop Flow ID because info is in other fields
data.drop([' Timestamp'], axis=1, inplace=True) # drop timestamp as we have them in order, not necessary

for column in data.columns:
    if data[column].dtypes == np.int64:
        data[column] = data[column].astype(np.float64)
    elif data[column].dtypes == np.float64:
        break
    else:
        for count, item in enumerate(data[column].values):
            data[column].values[count] = np.float64(int(ipaddress.IPv4Address(item)))
        data[column] = data[column].astype(np.float64)

# - - - - - - - - - -
# Scale the data

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() 

columns = data.columns[:-1]

data[columns] = scaler.fit_transform(data[columns])

# - - - - - - - - - -
# Split the data into 80% training, 20% testing
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(data, test_size=0.2, random_state=1)

# - - - - - - - - - -
# Here we create x_train, x_test, y_train, y_test as well as oversampling/undersampling data
# due to the large difference in benign and other data

print(df_train[' Label'].value_counts())
count_class_1, count_class_0 = df_train[' Label'].value_counts()

# divide df_train
df_class_0 = df_train[df_train[' Label'] == 0]
df_class_1 = df_train[df_train[' Label'] == 1]

print(df_class_0[' Label'].value_counts())
print(df_class_1[' Label'].value_counts())

# Oversampling
df_class_0_oversample = df_class_0.sample(round(count_class_1 / 10), replace=True)

# Undersampling
size_to_reduce_1_to = round(count_class_1 / 10)
df_class_1_undersample = df_class_1.sample(size_to_reduce_1_to)
count_class_1 = size_to_reduce_1_to

df_train_over_under = pd.concat([df_class_1_undersample, df_class_0_oversample], axis=0)
df_train = df_train_over_under

labels = df_train.columns[:-1]
x_train = df_train[labels]
y_train = df_train[' Label']

x_test = df_test[labels]
y_test = df_test[' Label']

print('Random combined-sampling:')
print(df_train_over_under[' Label'].value_counts())

In [ ]:
from sklearn import svm
import sklearn.model_selection as model_selection

In [ ]:
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(x_train, y_train)

In [ ]:
y_pred = rbf.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score

print("- - - Known Data - - -")
r2_value = r2_score(y_test, y_pred.round())
print(r2_value)

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, y_pred.round()))

In [ ]:
del data

In [ ]:
# Loading and formatting the unknown (UDPLag) data in the same was as the training data

udplagData = pd.DataFrame()

for chunk in pd.read_csv("data/03-11/UDPLag.csv", chunksize=chunksize, nrows=1000000):
    udplagData = udplagData.append(chunk)

udplagData.replace([np.inf, -np.inf], np.nan, inplace=True)
udplagData = udplagData.dropna()

import ipaddress

print(udplagData[' Label'].value_counts())

udplagData.replace({'UDP': 1, 'UDPLag': 1, 'Syn': 1, 'BENIGN': 0}, inplace=True)
udplagData[' Label'] = udplagData[' Label'].astype(np.float64)

udplagData['SimillarHTTP'] = udplagData['SimillarHTTP'].astype(bool).astype(np.float64)

udplagData.drop(['Unnamed: 0'], axis=1, inplace=True)
udplagData.drop(['Flow ID'], axis=1, inplace=True)
udplagData.drop([' Timestamp'], axis=1, inplace=True)

for column in udplagData.columns:
    if udplagData[column].dtypes == np.int64:
        udplagData[column] = udplagData[column].astype(np.float64)
    elif udplagData[column].dtypes == np.float64:
        break
    else:
        for count, item in enumerate(udplagData[column].values):
            udplagData[column].values[count] = np.float64(int(ipaddress.IPv4Address(item)))
        udplagData[column] = udplagData[column].astype(np.float64)


scaler = StandardScaler() 
columns = udplagData.columns[:-1]
udplagData[columns] = scaler.fit_transform(udplagData[columns])

x_test_udplag = udplagData[labels]
y_test_udplag = udplagData[' Label']

In [ ]:
y_pred = rbf.predict(x_test_udplag)

In [ ]:
print("- - - Unknown Data - - -")
r2_value = r2_score(y_test_udplag, y_pred_udplag.round())
print(r2_value)

In [ ]:
print(confusion_matrix(y_test_udplag, y_pred.round()))